In [23]:
import mysql.connector
import pandas as pd
import csv
import numpy as np


In [24]:
connection = mysql.connector.connect(user='root', password='Dovmf123')
cursor = connection.cursor(buffered=True)
data_rent = []
data_sale = []

#write rent data ################3
query = ("use centris_data_commercial_rent")
cursor.execute(query)

    
query = ("select * from features")
cursor.execute(query)
feature_title = ['Lot area','Zoning', 'Use of property', 'Building style', 'Year built', 'Fireplace/Stove', 'Number of units', 
            'Residential units', 'Main unit', 'Potential gross revenue', 'Additional features', 'Building area', 'Parking', 
             'Available area', 'Type of business', 'Pool', 'Type of operation', 'Residential unit', 'Intergenerational']

rent_feature_number = 16
sale_feature_number = 19

for i in cursor:
    index= 0
    for j in range(0,len(feature_title)):
        if (feature_title[j] == i[2]):
            index = j
    k = [int(i[0]), int(i[1]), index + 1, i[3]]
    data_rent.append(k)
    
data_rent.sort(key=lambda data_rent: data_rent[1])


data_rent = np.asarray(data_rent)

unique_elements = np.unique(np.transpose(data_rent)[1])

entry = []

for i in range(0,len(unique_elements)):
    l = ['' for i in range(0,rent_feature_number)]
    toadd = [unique_elements[i]]
    toadd.extend(l)
    entry.append(toadd)    
    
change = 0
mls = data_rent[0][1]
k = 0
for i in range(0,len(data_rent)):
    if (data_rent[i][1] != mls):
        k += 1
        mls = data_rent[i][1]
    entry[k][int(data_rent[i][2])] = data_rent[i][3] 
    
temp = ['mls_number']
temp.extend(feature_title[:16])
data_rent_final = np.append(np.asarray([temp]),entry, axis=0) 

data_rent_final = pd.DataFrame(data_rent_final)
data_rent_final.to_csv("rent_by_mls.csv", index=False)        


#write sale data    ############
query = ("use centris_data_commercial_sale")
cursor.execute(query)


query = ("select * from features")
cursor.execute(query)


for i in cursor:
    index= 0
    k = [int(i[0]), int(i[1]), int(i[2]), i[3]]
    data_sale.append(k)
    
data_sale.sort(key=lambda data_sale: data_sale[1])

data_sale = np.asarray(data_sale)

unique_elements = np.unique(np.transpose(data_sale)[1])

entry2 = []

for i in range(0,len(unique_elements)):
    l = [' ' for i in range(0,sale_feature_number)]
    toadd = [unique_elements[i]]
    toadd.extend(l)
    entry2.append(toadd)    
    
change = 0
mls = data_sale[0][1]
k = 0
for i in range(0,len(data_sale)):
    if (data_sale[i][1] != mls):
        k += 1
        mls = data_sale[i][1]

    entry2[k][int(data_sale[i][2])] = data_sale[i][3] 
temp = ['mls_number']
temp.extend(feature_title)
data_sale_final_array = np.append(np.asarray([temp]),entry2, axis=0) 
data_sale_final = pd.DataFrame(data_sale_final_array)
data_sale_final.to_csv("sale_by_mls.csv", index=False)        

      
 

In [25]:


complete_data_set = []
for i in data_sale_final_array[1:]:
    filled = 0
    if (i[1] == ' ') or (i[2] == ' ' and i[3] == ' ' ) or (i[5] == ' ') or (i[9] == ' ') or (i[10] == ' '):
        filled +=1 
    if filled == 0:
        if i[2] == ' ':
            complete_data_set.append([i[1], i[3], i[5], i[9], i[10]])
        else:
            complete_data_set.append([i[1], i[2], i[5], i[9], i[10]])
numerical_years = []

for i in complete_data_set:
    if i[2][:4].isnumeric():
        numerical_years.append(int(i[2][:4]))


average = int(sum(numerical_years)/len(numerical_years))
            
        

In [26]:
temp = []
for i in complete_data_set:
    q = i
    q[0]= q[0].replace(',', '')[:-5]
    q[4]= q[4].replace(',', '')[1:]
    if ('commercial' in q[1]):
        q[1] = 1
    else:
        q[1] = 0
    if 'Unknown' in i[2]:
        q[2] = str(average)
    elif 'New' in i[2]:
        q[2] = 2017
    else:
        q[2] = i[2][:4]
        
    index1 = 0
    index2 = 0
    index3 = 0         
        
    if 'Rooms' not in q[3]:
        index1 += 1
        continue
    if 'Bedrooms' not in q[3]:
        index2 += 1
        continue
    if 'Bath/powder' not in q[3]:
        index3 += 1
        continue
        
    string = q[3].split(',')


    string = [''.join(i for i in s if i.isdigit()) for s in string]
    q[3] = string
    if index1 == 1:
        k = [q[0], q[1], q[2], 0, q[3][1], q[3][2], q[4]]
    elif index2 == 1:
        k = [q[0], q[1], q[2], q[3][0], 0, q[3][2] , q[4]]
    elif index3 == 1 :
        k = [q[0], q[1], q[2], q[3][0], q[3][1], 0, q[4]]
    else:
        k = [q[0], q[1], q[2], q[3][0], q[3][1], q[3][2], q[4]]
    temp.append(k)


[sqft, commercial, year built, Room, bedrom, bath/powder, predicted value]

In [27]:
vector = pd.DataFrame(temp)
vector.to_csv("vector.csv", index=False, index_label = False)        


0        6
1        5
2        5
3       10
4        4
5        4
6       11
7        4
8        4
9        5
10       5
11       5
12      10
13       6
14       7
15      11
16       5
17       7
18       6
19       4
20       4
21       5
22       4
23       6
24       5
25      10
26       4
27       7
28       5
29       4
        ..
3395    10
3396     4
3397     6
3398     6
3399     5
3400     5
3401     6
3402     4
3403     7
3404     5
3405     4
3406     4
3407     7
3408     5
3409     4
3410     4
3411     4
3412     6
3413     8
3414    20
3415     4
3416     6
3417     4
3418     4
3419    12
3420     4
3421     5
3422     5
3423     5
3424     4
Name: 3, Length: 3425, dtype: object


In [28]:
complete_data_set = []
for i in data_sale_final_array[1:]:
    filled = 0
    if (i[1] == ' ') or (i[2] == ' ' and i[3] == ' ' ) or (i[5] == ' ') or (i[9] == ' ') or (i[10] == ' '):
        filled +=1 
    if filled == 0:
        if i[2] == ' ':
            complete_data_set.append([i[1], i[3], i[5], i[9], i[10]])
        else:
            complete_data_set.append([i[1], i[2], i[5], i[9], i[10]])
numerical_years = []

for i in complete_data_set:
    if i[2][:4].isnumeric():
        numerical_years.append(int(i[2][:4]))


temp = []
for i in complete_data_set:
    q = i
    q[0]= q[0].replace(',', '')[:-5]
    q[4]= q[4].replace(',', '')[1:]
    if ('commercial' in q[1]):
        q[1] = 1
    else:
        q[1] = 0
    if 'Unknown' in i[2]:
        q[2] = ''
    elif 'New' in i[2]:
        q[2] = 2017
    else:
        q[2] = i[2][:4]
        
    index1 = 0
    index2 = 0
    index3 = 0         
        
    if 'Rooms' not in q[3]:
        index1 += 1
        continue
    if 'Bedrooms' not in q[3]:
        index2 += 1
        continue
    if 'Bath/powder' not in q[3]:
        index3 += 1
        continue
        
    string = q[3].split(',')


    string = [''.join(i for i in s if i.isdigit()) for s in string]
    q[3] = string
    if index1 == 1:
        k = [q[0], q[1], q[2], 0, q[3][1], q[3][2], q[4]]
    elif index2 == 1:
        k = [q[0], q[1], q[2], q[3][0], 0, q[3][2] , q[4]]
    elif index3 == 1 :
        k = [q[0], q[1], q[2], q[3][0], q[3][1], 0, q[4]]
    else:
        k = [q[0], q[1], q[2], q[3][0], q[3][1], q[3][2], q[4]]
    temp.append(k)


In [29]:
vector2 = pd.DataFrame(temp)
vector2.to_csv("vector2.csv", index=False, index_label = False)        
